In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import re

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install implicit==0.4.4
# conda install -c conda-forge implicit

# # Функции из 1-ого вебинара
# import os, sys
# from metrics import precision_at_k, recall_at_k

In [3]:
!ls /content/drive/MyDrive/Colab Notebooks/Рекомендательные системы/metrics.py
!cat '/content/drive/MyDrive/Colab Notebooks/Рекомендательные системы/metrics.py'

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Рекомендательные системы/')

from metrics import precision_at_k, recall_at_k

ls: cannot access '/content/drive/MyDrive/Colab': No such file or directory
ls: cannot access 'Notebooks/Рекомендательные': No such file or directory
ls: cannot access 'системы/metrics.py': No such file or directory
"""
Metrics

"""
import numpy as np

def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)

def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

def money_precision_at_k(recommended_list, bought_list, prices_recom

In [4]:
# !pip install implicit==0.4.4

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Рекомендательные системы/retail_train.csv')
result = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Рекомендательные системы/predictions_basic.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### **Задание 1. Weighted Random Recommendation**

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [7]:
popularity = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popularity['weight'] = popularity['sales_value'] / popularity['sales_value'].sum()
popularity.head(2)

,item_id,sales_value,weight
0,25671,20.94,2.969296e-06
1,26081,0.99,1.403822e-07


In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=weights, replace=False)
    
    return recs.tolist()

In [9]:
%%time

# your_code

items_weights = popularity[['item_id', 'weight']]

result_weighted = pd.DataFrame()
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 3.71 s, sys: 107 ms, total: 3.82 s
Wall time: 4.69 s


### **Задание 2. Расчет метрик**
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [10]:
# функция для преобразования строк в список
def str_to_array(result):
    shape_ = result.shape[0]
    for col in result.columns[1:]:        
        for string in range(shape_):
            if isinstance(result[col].iloc[string], str):
                result[col].iloc[string] = [int(_) for _ in re.findall(r'-?\d+\.?\d*', result[col].iloc[string])]
            else:
                result[col].iloc[string] = result[col].iloc[string]

    return result

In [11]:
str_to_array(result).head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1120297, 12672919, 865377, 14074566, 1073212]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[13008227, 5571134, 10355675, 1075313, 969932]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12985723, 13511470, 976405, 6920289, 953154]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[962229, 15927953, 871876, 13512542, 1078830]"


In [12]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:  {name_col}")

1.0:  actual
0.0008:  random_recommendation
0.1552:  popular_recommendation
0.1369:  itemitem
0.1329:  cosine
0.139:  tfidf
0.2019:  own_purchases
0.0217:  weighted_random_recommendation


**Вывод:** наилучшая метрика получается для 'own_purchases' (собственных покупок) - $0.2019$, в связи с тем, что покупатель с определенной частотой берет одинаковый набор продуктов через какой-то промежуток времени (офлайн покупки). Соответственно вероятность, что мы угадаем правильную рекомендацию, высокая.

### **Задание 3*. Улучшение бейзлайнов и ItemItem**

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [13]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [14]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [15]:
# отберем топ-5000
quant_sum = data_train.groupby('item_id')['quantity'].sum().reset_index()
quant_sum.sort_values(by='quantity', ascending=False, inplace=True)
top_5000 = quant_sum.head(5000)

In [16]:
%%time

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(top_5000['item_id'], n=5))
result.head(2)

CPU times: user 438 ms, sys: 2.69 ms, total: 441 ms
Wall time: 606 ms


In [17]:
%%time

data_train_top5000 = data_train.loc[data_train['item_id'].isin(top_5000['item_id']), :]
result['popular_recommendation_5000'] = result['user_id'].apply(lambda x: popularity_recommendation(data_train_top5000, n=5))
result.head(2)

CPU times: user 1min 10s, sys: 2.56 s, total: 1min 13s
Wall time: 1min 14s


In [18]:
data_train_top5000.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [19]:
popularity_5000 = data_train_top5000.groupby('item_id')['sales_value'].sum().reset_index()
popularity_5000['weight'] = popularity_5000['sales_value'] / popularity_5000['sales_value'].sum()
popularity_5000.head(2)

,item_id,sales_value,weight
0,202291,81.59,0.000020
1,397896,2862.41,0.000706


In [20]:
%%time

items_weights = popularity_5000[['item_id', 'weight']]
result['weighted_random_recommendation_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: user 336 ms, sys: 3 ms, total: 339 ms
Wall time: 340 ms


In [21]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:  {name_col}")

1.0:  actual
0.0008:  random_recommendation
0.1552:  popular_recommendation
0.1369:  itemitem
0.1329:  cosine
0.139:  tfidf
0.2019:  own_purchases
0.0217:  weighted_random_recommendation
0.0052:  random_recommendation_5000
0.1552:  popular_recommendation_5000
0.0363:  weighted_random_recommendation_5000


In [22]:
compare_table_5000 = pd.DataFrame({
    'precision_at_k': [0.0006, 0.006, 0.1552, 0.1552, 0.0212, 0.0374],
    'name_col': ['random_recommendation', 'random_recommendation_5000', 'popular_recommendation', 'popular_recommendation_5000', \
                 'weighted_random_recommendation', 'weighted_random_recommendation_5000']   
})

compare_table_5000

,precision_at_k,name_col
0,0.0006,random_recommendation
1,0.0060,random_recommendation_5000
2,0.1552,popular_recommendation
3,0.1552,popular_recommendation_5000
4,0.0212,weighted_random_recommendation
5,0.0374,weighted_random_recommendation_5000


In [23]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000['item_id']), 'item_id'] = 999999
data_train.head(100)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,999999,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [24]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()


userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [25]:
%%time

k_num = [2, 3, 4, 5]
result_recs = []

for i in k_num:
    
    model = ItemItemRecommender(K=i, num_threads=2) # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=False)
    recs = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])
    
    result_recs.append(recs)
    
    
result['itemitem_2'] = result_recs[0]
result['itemitem_3'] = result_recs[1]
result['itemitem_4'] = result_recs[2]
result['itemitem_5'] = result_recs[3]

CPU times: user 7.79 s, sys: 21.2 ms, total: 7.81 s
Wall time: 5.7 s


In [26]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:  {name_col}")

1.0:  actual
0.0008:  random_recommendation
0.1552:  popular_recommendation
0.1369:  itemitem
0.1329:  cosine
0.139:  tfidf
0.2019:  own_purchases
0.0217:  weighted_random_recommendation
0.0052:  random_recommendation_5000
0.1552:  popular_recommendation_5000
0.0363:  weighted_random_recommendation_5000
0.192:  itemitem_2
0.1861:  itemitem_3
0.145:  itemitem_4
0.1369:  itemitem_5


**Вывод:** Метрики для **бейзлайнов** удалось улучшить, считая их на топ-5000 товаров (например, $0.0247$:  weighted_random_recommendation и $0.0352$:  weighted_random_recommendation_5000), кроме расчета популярных товаров, в данном случае метрика не изменилась ($0.1552$).

Для метода **ItemItem** наилучшая метрика получилась с количеством соседей $k=2$ ($0.192$).